In [3]:
import torch as tc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import processing.utils as my
from torch.utils import data
from torch import nn
from tqdm import tqdm
import plotly

In [4]:
batch_size = 128
epochs = 10
mydata = pd.read_csv('data.csv')
CI = my.get_CI(mydata)
for i in np.argmax(np.asarray(CI), axis=0):
    print(i)
    CI.pop(i)
CI = np.asarray(CI)
features, labels = CI[:, 0:2], CI[:, 2]
features, labels = tc.tensor(features, dtype=tc.float), tc.tensor(labels, dtype=tc.float).unsqueeze(1)
labels = tc.log(labels)
features = tc.log(features)

817
766
499


In [5]:
# features = tc.linspace(0, 10, 1000)
# labels = tc.sin(features) + tc.rand(1000)*0.2
# features, labels = features.unsqueeze(1), labels.unsqueeze(1)

In [6]:
features.shape, labels.shape

(torch.Size([1683, 2]), torch.Size([1683, 1]))

In [7]:
def load_array(array, batchsize, is_train=True):
    dataset = data.TensorDataset(*array)
    return data.DataLoader(dataset, batch_size=batchsize, shuffle=is_train)
dataloader = load_array((features, labels), batch_size)

In [8]:
ks1 = [np.sin(1)- np.sin(0)]
ks2 = [1.0]
bias = [0]
for i in range(2, 11):
    k = np.sin(i)- np.sin(i-1)
    k_ = k - sum(np.array(ks1)*np.array(ks2))
    if k_ < 0:
        ks2.append(-1.0)
        ks1.append(-k_)
    else:
        ks2.append(1.0)
        ks1.append(k_)
    bias.append(ks1[-1]*(1-i))

In [9]:
net = nn.Sequential(nn.Linear(2, 1024),
                    nn.Dropout(),
                    nn.ReLU(),
                    nn.Linear(1024, 2056),
                    nn.Dropout(),
                    nn.ReLU(),
                    nn.Linear(2056, 1024),
                    nn.Dropout(),
                    nn.ReLU(),
                    nn.Linear(1024, 1))
trainer = tc.optim.SGD(net.parameters(), lr=0.03)
loss = nn.MSELoss()
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

Sequential(
  (0): Linear(in_features=2, out_features=1024, bias=True)
  (1): Dropout(p=0.5, inplace=False)
  (2): ReLU()
  (3): Linear(in_features=1024, out_features=2056, bias=True)
  (4): Dropout(p=0.5, inplace=False)
  (5): ReLU()
  (6): Linear(in_features=2056, out_features=1024, bias=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): ReLU()
  (9): Linear(in_features=1024, out_features=1, bias=True)
)

In [10]:
# net[0].weight.data = tc.tensor(ks1, dtype=tc.float).unsqueeze(1)
# net[0].bias.data = tc.tensor(bias, dtype=tc.float)
# net[2].weight.data = tc.tensor(ks2, dtype=tc.float).unsqueeze(0)
# net[2].bias.data = tc.tensor([0.0 for i in range(1)], dtype=tc.float)

In [11]:
# net[0].weight.data = tc.tensor([1 for i in range(10)], dtype=tc.float).unsqueeze(1)
# net[0].bias.data = tc.tensor([0.0 for i in range(10)], dtype=tc.float)
# net[2].weight.data = tc.tensor([0 for i in range(10)], dtype=tc.float).unsqueeze(0)
# net[2].bias.data = tc.tensor([0.0 for i in range(1)], dtype=tc.float)

In [12]:
# net[2].weight.data

In [13]:
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for X, y in dataloader:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
print(f'epoch {epoch + 1}, loss {l:f}')

100%|██████████| 10/10 [00:01<00:00,  8.55it/s]

epoch 10, loss 0.079934


In [14]:
l = loss(net(features), labels)
print(f'epoch {epoch + 1}, loss {l:f}')
import plotly.graph_objects as go
net.train(False)
y_ = net(features)
x = y_.flatten().detach()
y = labels.flatten().detach()
fig = go.Figure()
# fig.add_trace(go.Scatter(x = features.flatten(), y=labels.flatten(), mode='markers'))
# fig.add_trace(go.Scatter(x = features.flatten(), y= x))
fig.add_trace(go.Scatter(x = x, y=y, mode='markers'))

# 显示图表
fig.show()

epoch 10, loss 0.080047
